In [1]:
# Install BigQuery client library (run this cell once)
%pip install google-cloud-bigquery --quiet
%pip install bigquery-magics --quiet
%pip install db-dtypes --quiet
# Import libraries
from google.cloud import bigquery
import pandas as pd 
%load_ext bigquery_magics
import db_dtypes
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "salesforce-465614-2cf9e37da64b.json"
from google.cloud import bigquery
# Initialize BigQuery client
client = bigquery.Client()

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%%bqsql
 select * from salesforce-465614.cust_analytics.report_transactions limit 2

Query is running:   0%|          |

Downloading:   0%|          |

,transaction_id,customer_id,product_id,Store_id,order_status,Purchase_Date,category,sub_category,Unit_Price,quantity,Total_Price,payment_method,channel,loyalty_program,age,gender,city
0,T98509,C1330,P419,STORE_7,Completed,2024-03-07,Electronics,Accessories,113.77,3,341.31,Bank Transfer,Physical,true,20,Female,Port Jeromeshire
1,T517643,C1952,P805,STORE_4,Cancelled,2024-03-07,Toys,Board Games,207.78,2,415.56,Bank Transfer,Physical,false,66,Male,Williamsland


In [4]:
%%bqsql
SELECT 
  category,
  channel,
  SUM(total_price) AS total_revenue,
  COUNT(*) AS transaction_count
FROM `salesforce-465614.cust_analytics.report_transactions`
WHERE order_status = 'Completed'
GROUP BY category, channel
ORDER BY total_revenue DESC
limit 5


Query is running:   0%|          |

Downloading:   0%|          |

,category,channel,total_revenue,transaction_count
0,Books,Physical,5651307.75,1972
1,Beauty,Physical,4672627.00,1706
2,Electronics,Physical,4256474.44,1405
3,Clothing,Physical,4091927.72,1632
4,Toys,Physical,3193546.68,1180


In [6]:
%%bqsql
    SELECT 
  gender,
  loyalty_program,
  CASE 
    WHEN age < 25 THEN 'Under 25'
    WHEN age BETWEEN 25 AND 34 THEN '25-34'
    WHEN age BETWEEN 35 AND 44 THEN '35-44'
    WHEN age BETWEEN 45 AND 54 THEN '45-54'
    ELSE '55+' 
  END AS age_group,
  COUNT(*) AS purchase_count,
  SUM(total_price) AS total_spent
FROM `salesforce-465614.cust_analytics.report_transactions`
WHERE order_status = 'Completed'
GROUP BY gender, loyalty_program, age_group
ORDER BY total_spent DESC
limit 10


Query is running:   0%|          |

Downloading:   0%|          |

,gender,loyalty_program,age_group,purchase_count,total_spent
0,Male,true,55+,2367,6587862.31
1,Female,true,55+,1655,4594675.71
2,Male,false,55+,1019,2777286.20
3,Male,true,35-44,908,2542308.96
4,Male,true,25-34,919,2506794.31
5,Male,true,45-54,854,2391709.80
6,Female,true,45-54,630,1754027.33
7,Female,false,55+,605,1735068.99
8,Male,true,Under 25,630,1705714.24
9,Female,true,35-44,577,1658710.10


In [ ]:
%%bqsql
 SELECT 
  channel,
  COUNT(*) AS total_transactions,
  round(AVG(total_price),2) AS avg_order_value,
  SUM(total_price) AS total_sales
FROM `salesforce-465614.cust_analytics.report_transactions`
WHERE order_status = 'Completed'
GROUP BY channel;

Query is running:   0%|          |

Downloading:   0%|          |

,channel,total_transactions,avg_order_value,total_sales
0,Online,4478,2763.17,12373485.93
1,Physical,8954,2791.26,24992961.94


In [11]:
%%bqsql
WITH customer_spend AS (
  SELECT 
    customer_id,
    SUM(total_price) AS total_spent
  FROM `salesforce-465614.cust_analytics.report_transactions`
  WHERE order_status = 'Completed'
  GROUP BY customer_id
),
ranked_customers AS (
  SELECT 
    customer_id,
    total_spent,
    NTILE(10) OVER (ORDER BY total_spent DESC) AS decile
  FROM customer_spend
)
SELECT 
  customer_id,
  total_spent
FROM ranked_customers
WHERE decile = 1
ORDER BY total_spent DESC
limit 10


Query is running:   0%|          |

Downloading:   0%|          |

,customer_id,total_spent
0,C315,57776.33
1,C1737,57531.13
2,C983,54505.20
3,C1270,52783.92
4,C1226,52173.05
5,C1347,49145.28
6,C1422,47871.33
7,C431,47264.73
8,C616,47093.54
9,C929,47009.10


In [13]:
%%bqsql
SELECT 
  order_status,
  COUNT(*) AS order_count,
  SUM(total_price) AS total_value
FROM `salesforce-465614.cust_analytics.report_transactions`
GROUP BY order_status
ORDER BY order_count DESC;


Query is running:   0%|          |

Downloading:   0%|          |

,order_status,order_count,total_value
0,Completed,13432,37366447.87
1,Cancelled,6568,18545186.49


In [20]:
%%bqsql
SELECT 
  customer_id,
  DATE_DIFF(date('2024-09-24'), MAX(purchase_date), DAY) AS recency_days
FROM `salesforce-465614.cust_analytics.report_transactions`
WHERE order_status = 'Completed'
GROUP BY customer_id
order by 2  limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,customer_id,recency_days
0,C815,1
1,C1421,1
2,C1257,1
3,C1822,1
4,C256,1
5,C1127,1
6,C1735,1
7,C299,1
8,C1169,1
9,C284,1


In [22]:
%%bqsql
SELECT 
  customer_id,
  COUNT(*) AS frequency,
  SUM(total_price) AS monetary
FROM `salesforce-465614.cust_analytics.report_transactions`
WHERE order_status = 'Completed'
GROUP BY customer_id
order by 2 desc limit 10;


Query is running:   0%|          |

Downloading:   0%|          |

,customer_id,frequency,monetary
0,C315,15,57776.33
1,C651,15,34439.83
2,C1226,15,52173.05
3,C1716,15,26711.96
4,C1232,15,36490.19
5,C1651,14,34717.01
6,C431,14,47264.73
7,C1670,14,39171.39
8,C1114,14,46223.07
9,C1270,14,52783.92


In [ ]:
%%bqsql
SELECT 
  customer_id,
  DATE_DIFF(date('2024-09-24'), MAX(purchase_date), DAY) AS recency_days,
  COUNT(*) AS frequency,
  SUM(total_price) AS monetary
FROM `salesforce-465614.cust_analytics.report_transactions`
WHERE order_status = 'Completed'
GROUP BY customer_id
order by 2, 3 desc
limit 10;

Query is running:   0%|          |

Downloading:   0%|          |

,customer_id,recency_days,frequency,monetary
0,C1670,1,14,39171.39
1,C1482,1,11,27134.22
2,C1790,1,10,29725.95
3,C960,1,10,18172.30
4,C860,1,10,35283.59
5,C1859,1,10,31524.75
6,C1257,1,10,35389.16
7,C1735,1,9,18960.75
8,C333,1,9,23258.66
9,C866,1,9,25451.32


In [29]:
%%bqsql
SELECT 
  customer_id,

  -- Recency in days
  DATE_DIFF(date('2024-09-24'), MAX(purchase_date), DAY) AS recency_days,

  -- Recency bucket
  CASE 
    WHEN DATE_DIFF(date('2024-09-24'), MAX(purchase_date), DAY)  <= 30 THEN 'Recent'
    WHEN DATE_DIFF(date('2024-09-24'), MAX(purchase_date), DAY) <= 90 THEN 'Moderate'
    ELSE 'Stale'
  END AS recency_bucket,

  -- Frequency count
  COUNT(*) AS frequency,

  -- Frequency bucket
  CASE 
    WHEN COUNT(*) >= 10 THEN 'Frequent'
    WHEN COUNT(*) >= 5 THEN 'Moderate'
    ELSE 'Infrequent'
  END AS frequency_bucket,

  -- Monetary total
  SUM(total_price) AS monetary,

  -- Monetary bucket
  CASE 
    WHEN SUM(total_price) >= 10000 THEN 'High Spender'
    WHEN SUM(total_price) >= 5000 THEN 'Medium Spender'
    ELSE 'Low Spender'
  END AS monetary_bucket

FROM `salesforce-465614.cust_analytics.report_transactions`
WHERE order_status = 'Completed'
GROUP BY customer_id;


Query is running:   0%|          |

Downloading:   0%|          |

,customer_id,recency_days,recency_bucket,frequency,frequency_bucket,monetary,monetary_bucket
0,C1019,10,Recent,8,Moderate,15136.78,High Spender
1,C1824,160,Stale,4,Infrequent,13560.77,High Spender
2,C858,66,Moderate,7,Moderate,25225.38,High Spender
3,C750,7,Recent,9,Moderate,24929.70,High Spender
4,C961,36,Moderate,10,Frequent,22957.21,High Spender
...,...,...,...,...,...,...,...
1992,C1771,63,Moderate,6,Moderate,16940.93,High Spender
1993,C1511,29,Recent,6,Moderate,27628.96,High Spender
1994,C1763,35,Moderate,6,Moderate,20849.35,High Spender
1995,C803,152,Stale,3,Infrequent,5604.12,Medium Spender


In [34]:
%%bqsql
SELECT 
  customer_id,

  -- Recency in days
  DATE_DIFF(date('2024-09-24'), MAX(purchase_date), DAY) AS recency_days,

  -- Frequency
  COUNT(*) AS frequency,

  -- Monetary
  SUM(total_price) AS monetary,

  -- Recency bucket
  CASE 
     WHEN
    DATE_DIFF(date('2024-09-24'), MAX(purchase_date), DAY) <= 30 THEN 'Recent'
    when DATE_DIFF(date('2024-09-24'), MAX(purchase_date), DAY) <= 90 THEN 'Moderate'
    ELSE 'Stale'
  END AS recency_bucket,

  -- Frequency bucket
  CASE 
    WHEN COUNT(*) >= 10 THEN 'Frequent'
    WHEN COUNT(*) >= 5 THEN 'Moderate'
    ELSE 'Infrequent'
  END AS frequency_bucket,

  -- Monetary bucket
  CASE 
    WHEN SUM(total_price) >= 10000 THEN 'High Spender'
    WHEN SUM(total_price) >= 5000 THEN 'Medium Spender'
    ELSE 'Low Spender'
  END AS monetary_bucket,

  CASE
    WHEN DATE_DIFF(date('2024-09-24'), MAX(purchase_date), DAY) <= 30 
         AND COUNT(*) >= 10 
         AND SUM(total_price) >= 10000 THEN 'Loyal Champion'

    WHEN DATE_DIFF(date('2024-09-24'), MAX(purchase_date), DAY) <= 30 
         AND COUNT(*) BETWEEN 5 AND 9 
         AND SUM(total_price) >= 5000 THEN 'High Potential'

    WHEN DATE_DIFF(date('2024-09-24'), MAX(purchase_date), DAY) BETWEEN 31 AND 90 
         AND COUNT(*) <= 4 
         AND SUM(total_price) < 5000 THEN 'At Risk'

    WHEN DATE_DIFF(date('2024-09-24'), MAX(purchase_date), DAY) > 90 
         AND COUNT(*) < 3 THEN 'Churned'

    WHEN COUNT(*) >= 10 
         AND SUM(total_price) >= 10000 THEN 'Big Spender'

    WHEN DATE_DIFF(date('2024-09-24'), MAX(purchase_date), DAY) <= 30 
         AND COUNT(*) < 3 THEN 'New Customer'

    ELSE 'Other'
  END AS customer_segment

FROM `salesforce-465614.cust_analytics.report_transactions`
WHERE order_status = 'Completed'
GROUP BY customer_id;


Query is running:   0%|          |

Downloading:   0%|          |

,customer_id,recency_days,frequency,monetary,recency_bucket,frequency_bucket,monetary_bucket,customer_segment
0,C1292,108,7,19571.03,Stale,Moderate,High Spender,Other
1,C1890,15,11,27209.34,Recent,Frequent,High Spender,Loyal Champion
2,C1155,44,11,23551.29,Moderate,Frequent,High Spender,Big Spender
3,C1670,1,14,39171.39,Recent,Frequent,High Spender,Loyal Champion
4,C1830,7,10,30903.87,Recent,Frequent,High Spender,Loyal Champion
...,...,...,...,...,...,...,...,...
1992,C564,28,6,23195.33,Recent,Moderate,High Spender,High Potential
1993,C1900,104,5,18921.54,Stale,Moderate,High Spender,Other
1994,C657,21,4,10936.64,Recent,Infrequent,High Spender,Other
1995,C286,78,4,9896.93,Moderate,Infrequent,Medium Spender,Other
